<a href="https://colab.research.google.com/github/vgomesfalcao/planilha_central/blob/main/Capital_planilha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
pd.set_option('max_colwidth', None)

In [ ]:
planilha_dexion = 'itau_convertido.xlsx'
planilha_sygecom = 'Planilha Excel Sygecom.xlsx'

In [ ]:
dexion = pd.read_excel(planilha_dexion)
dexion = dexion.drop(range(8))
dexion = dexion.drop(dexion.index[-1])
dexion = dexion.dropna(how='all', axis=1)
dexion.columns = ['Data', 'Lançamento','Contra-Partida','Parenteses','Atalho','Histórico','Valor de Débito','Valor de Crédito', 'Saldo','Deb/Cred']
dexion['Atalho'] = dexion['Parenteses'] + dexion['Atalho']
dexion = dexion.drop(columns=['Parenteses'])
dexion = dexion.reset_index(drop=True)

In [ ]:
for row in dexion.itertuples():
  if(pd.isnull(row.Data)):
    # display(row.Index-1)
    # break
    subindex = 1
    while True:
      if((row.Index-subindex) in dexion.index):
        linha_anterior = dexion.loc[[row.Index-subindex],'Histórico'].values[0]
        break
      else:
        subindex += 1
        
    dexion.loc[[row.Index-subindex],'Histórico'] =  linha_anterior + row.Histórico
    dexion = dexion.drop(row.Index)
dexion = dexion[dexion['Data'] != 'CENTRAL ORGANIZACAO CONTABIL S/C LTDA']
dexion = dexion.dropna(how='all', axis=0)
dexion = dexion.reset_index(drop=True)
dexion

In [ ]:
dexion['Cheques'] = dexion['Histórico'].str.extract(r'((?<=CH:)[\d,\s]{3,10})')
dexion['Cheques'] = dexion['Cheques'].str.replace(' ','')
dexion

In [ ]:
datatoexcel = pd.ExcelWriter('processado/cleaned_'+path)
dexion.to_excel(datatoexcel)
datatoexcel.save()

In [ ]:
sygecom = pd.read_excel(planilha_sygecom)
sygecom = sygecom.drop(columns=['Data Emi','Data Comp','Unnamed: 9','Pago/Rec P/Usuário','Unnamed: 18','Hora Lanc','Filial','Favorecido','Contato','Conta DEB','Conta CRE','CPF/CNPJ','Unnamed: 30','Invoice','Contrato','Adiantamento','Cód. Baixa'])
sygecom.rename(columns={'Data Venc':'Data','Saldo   R$':'Saldo','Cheque N.':'Cheques'},inplace = True)
sygecom = sygecom.dropna(how='all', axis=1)

In [ ]:
dexionna = dexion
dexionna[['Cheques']] = dexionna[['Cheques']].fillna(0)
dexionna['Cheques'] = dexionna['Cheques'].astype('int64')
dexionna['Data'] = pd.to_datetime(dexion['Data'],format='%d/%m/%Y')


In [ ]:
sygecom['Cheques'] = sygecom['Cheques'].astype('int64')
sygecom.info()

In [ ]:
dexionna.info()

In [ ]:
juncao = dexionna.append(sygecom).sort_values(['Data','Cheques'])
datatoexcel = pd.ExcelWriter('processado/cleaned_juncao.xlsx')
juncao.to_excel(datatoexcel)
datatoexcel.save()